In [1]:
#wallet to exchange
%matplotlib inline
import emoji
import matplotlib.pyplot as plt
import re
import pandas as pd
import glob
from tqdm import tqdm, tqdm_notebook
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import os
import math
from pprint import pprint
import quantstats
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# log btc for wtoe and etow

# wtoe : negative btc sentiment -> x -1
#etow : positive btc sentiment

#glob.glob

In [2]:
df_etow=pd.read_csv("cleaned_twitter_etow.csv")
df_etow=df_etow[['Datetime','BTC','USD']]
df_etow['BTC_change']=df_etow['BTC']*1
df_etow['USD_change']=df_etow['USD']*1
df_etow

,Datetime,BTC,USD,BTC_change,USD_change
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579
...,...,...,...,...,...
4532,2018-09-19 16:54:53+00:00,928,5891728,928,5891728
4533,2018-09-19 09:04:06+00:00,955,6083566,955,6083566
4534,2018-09-19 04:45:51+00:00,946,6012967,946,6012967
4535,2018-09-18 09:15:51+00:00,997,6269280,997,6269280


In [3]:
df_wtoe=pd.read_csv("cleaned_twitter_wtoe.csv")
df_wtoe=df_wtoe[['Datetime','BTC','USD']]
df_wtoe['BTC_change']=df_wtoe['BTC']*(-1)
df_wtoe['USD_change']=df_wtoe['USD']*(-1)
df_wtoe

,Datetime,BTC,USD,BTC_change,USD_change
0,2021-10-17 19:44:48+00:00,499,30215276,-499,-30215276
1,2021-10-17 14:39:50+00:00,565,34505107,-565,-34505107
2,2021-10-16 05:33:55+00:00,829,51559960,-829,-51559960
3,2021-10-16 04:33:02+00:00,360,22408806,-360,-22408806
4,2021-10-15 23:26:51+00:00,700,42773593,-700,-42773593
...,...,...,...,...,...
2891,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999
2892,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335
2893,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703
2894,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815


In [4]:
df=pd.concat([df_etow, df_wtoe], ignore_index=True, sort=False)
df

,Datetime,BTC,USD,BTC_change,USD_change
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579
...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815


In [10]:
df['datetimeday']=pd.to_datetime(df['Datetime'], errors='coerce').dt.date
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m,datetime5m,datetimeday
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00,2021-10-17 21:30:00,2021-10-17
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00,2021-10-15 20:55:00,2021-10-15
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00,2021-10-15 17:55:00,2021-10-15
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00,2021-10-15 17:10:00,2021-10-15
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00,2021-10-14 19:10:00,2021-10-14
...,...,...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00,2018-09-21 02:30:00,2018-09-21
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00,2018-09-20 22:20:00,2018-09-20


In [6]:
#df.to_csv("final_twitter_sentiments.csv",index=False)

In [7]:
temp1=pd.to_datetime(df['Datetime'], errors='coerce').dt.floor('4h')
temp1=pd.to_datetime(temp1.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime4h']=temp1
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00
...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00


In [8]:
temp2=pd.to_datetime(df['Datetime'], errors='coerce').dt.floor('1min') 
temp2=pd.to_datetime(temp2.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime1m']=temp2
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00
...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00


In [9]:
temp3=pd.to_datetime(df['Datetime'], errors='coerce').dt.floor('5min') 
temp3=pd.to_datetime(temp3.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime5m']=temp3
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m,datetime5m
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00,2021-10-17 21:30:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00,2021-10-15 20:55:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00,2021-10-15 17:55:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00,2021-10-15 17:10:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00,2021-10-14 19:10:00
...,...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00,2018-09-21 02:30:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00,2018-09-20 22:20:00


In [11]:
temp4=pd.to_datetime(df['Datetime'], errors='coerce').dt.floor('15min') 
temp4=pd.to_datetime(temp4.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime15m']=temp4
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m,datetime5m,datetimeday,datetime15m
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00,2021-10-17 21:30:00,2021-10-17,2021-10-17 21:30:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00,2021-10-15 20:55:00,2021-10-15,2021-10-15 20:45:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00,2021-10-15 17:55:00,2021-10-15,2021-10-15 17:45:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00,2021-10-15 17:10:00,2021-10-15,2021-10-15 17:00:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00,2021-10-14 19:10:00,2021-10-14,2021-10-14 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00,2018-09-21 02:30:00,2018-09-21,2018-09-21 02:30:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00,2018-09-20 22:20:00,2018-09-20,2018-09-20 22:15:00


In [12]:
temp5=pd.to_datetime(df['Datetime'], errors='coerce').dt.floor('1h') 
temp5=pd.to_datetime(temp5.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime1h']=temp5
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m,datetime5m,datetimeday,datetime15m,datetime1h
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00,2021-10-17 21:30:00,2021-10-17,2021-10-17 21:30:00,2021-10-17 21:00:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00,2021-10-15 20:55:00,2021-10-15,2021-10-15 20:45:00,2021-10-15 20:00:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00,2021-10-15 17:55:00,2021-10-15,2021-10-15 17:45:00,2021-10-15 17:00:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00,2021-10-15 17:10:00,2021-10-15,2021-10-15 17:00:00,2021-10-15 17:00:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00,2021-10-14 19:10:00,2021-10-14,2021-10-14 19:00:00,2021-10-14 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00,2018-09-21 05:00:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00,2018-09-21 05:00:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00,2018-09-21 02:30:00,2018-09-21,2018-09-21 02:30:00,2018-09-21 02:00:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00,2018-09-20 22:20:00,2018-09-20,2018-09-20 22:15:00,2018-09-20 22:00:00


In [13]:
temp6=pd.to_datetime(df['Datetime'], errors='coerce').dt.floor('12h') 
temp6=pd.to_datetime(temp6.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))
df['datetime12h']=temp6
df

,Datetime,BTC,USD,BTC_change,USD_change,date,datetime4h,datetime1m,datetime5m,datetimeday,datetime15m,datetime1h,datetime12h
0,2021-10-17 21:34:18+00:00,645,38674244,645,38674244,2021-10-17,2021-10-17 20:00:00,2021-10-17 21:34:00,2021-10-17 21:30:00,2021-10-17,2021-10-17 21:30:00,2021-10-17 21:00:00,2021-10-17 12:00:00
1,2021-10-15 20:59:41+00:00,341,21397789,341,21397789,2021-10-15,2021-10-15 20:00:00,2021-10-15 20:59:00,2021-10-15 20:55:00,2021-10-15,2021-10-15 20:45:00,2021-10-15 20:00:00,2021-10-15 12:00:00
2,2021-10-15 17:59:04+00:00,500,30861741,500,30861741,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:59:00,2021-10-15 17:55:00,2021-10-15,2021-10-15 17:45:00,2021-10-15 17:00:00,2021-10-15 12:00:00
3,2021-10-15 17:11:18+00:00,881,54535823,881,54535823,2021-10-15,2021-10-15 16:00:00,2021-10-15 17:11:00,2021-10-15 17:10:00,2021-10-15,2021-10-15 17:00:00,2021-10-15 17:00:00,2021-10-15 12:00:00
4,2021-10-14 19:10:30+00:00,839,48322579,839,48322579,2021-10-14,2021-10-14 16:00:00,2021-10-14 19:10:00,2021-10-14 19:10:00,2021-10-14,2021-10-14 19:00:00,2021-10-14 19:00:00,2021-10-14 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7428,2018-09-21 05:06:41+00:00,949,6221999,-949,-6221999,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00,2018-09-21 05:00:00,2018-09-21 00:00:00
7429,2018-09-21 05:06:41+00:00,1000,6555335,-1000,-6555335,2018-09-21,2018-09-21 04:00:00,2018-09-21 05:06:00,2018-09-21 05:05:00,2018-09-21,2018-09-21 05:00:00,2018-09-21 05:00:00,2018-09-21 00:00:00
7430,2018-09-21 02:32:07+00:00,1500,9769703,-1500,-9769703,2018-09-21,2018-09-21 00:00:00,2018-09-21 02:32:00,2018-09-21 02:30:00,2018-09-21,2018-09-21 02:30:00,2018-09-21 02:00:00,2018-09-21 00:00:00
7431,2018-09-20 22:23:49+00:00,1229,7969815,-1229,-7969815,2018-09-20,2018-09-20 20:00:00,2018-09-20 22:23:00,2018-09-20 22:20:00,2018-09-20,2018-09-20 22:15:00,2018-09-20 22:00:00,2018-09-20 12:00:00


In [14]:
df.to_csv("final_twitter_sentiments.csv",index=False)